## Fashion data Bayesian Causal Forest

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, GridSearchCV

In [2]:
from xbcausalforest import XBCF

The data is the result of a simulation on top of some real data. The treatment effect on the purchase amount is nonlinear dependent on some X variables and lies roughly between -40 and 100. 

We are interested to estimate the treatment effect on the purchase/response `tau_response`.

In [3]:
X = pd.read_csv("../data/fashionB_clean_nonlinear.csv")
#X = data.copy()

# PARAMETERS
SEED=42
np.random.seed(SEED)

c = X.pop('converted')
z = X.pop('TREATMENT')
y = X.pop('checkoutAmount')
tau_conversion = X.pop('TREATMENT_EFFECT_CONVERSION')
tau_basket = X.pop('TREATMENT_EFFECT_BASKET')
tau_response = X.pop('TREATMENT_EFFECT_RESPONSE')

In [4]:
# Find columns that are not binary with max=1
num_columns = np.where(X.columns[(X.max(axis=0) != 1)])[0].tolist()

In [5]:
splitter = StratifiedKFold(n_splits=2, shuffle=True, random_state=SEED)
cz_groups = 2*z+c # Groups 0-4 depending on combinations [0,1]x[0,1]
folds = list(splitter.split(X, cz_groups))

In [6]:
# Split the train and validation data
split = folds[0]
X_val, y_val, c_val, z_val, tau_conversion_val, tau_basket_val, tau_response_val = [obj.to_numpy().astype(float)[split[1]] for obj in [X, y, c, z, tau_conversion, tau_basket, tau_response]]
X, y, c, z, tau_conversion, tau_basket, tau_response = [obj.to_numpy().astype(float)[split[0]] for obj in [X, y, c, z, tau_conversion, tau_basket, tau_response]]


# Normalize the data
ct = ColumnTransformer([
    # (name, transformer, columns)
    # Transformer for categorical variables
    #("onehot",
    #     OneHotEncoder(categories='auto', handle_unknown='ignore', ),
    #     cat_columns),
    # Transformer for numeric variables
    ("num_preproc", StandardScaler(), num_columns)
    ],
    # what to do with variables not specified in the indices?
    remainder="passthrough")

X = ct.fit_transform(X)
X_val = ct.transform(X_val)

In [7]:
# Treatment indicator 1/0 yes/no
z= z.astype('int32')

# scale response variable
meany = np.mean(y)
sdy = np.std(y)
y_norm = y - np.mean(y)
y_norm = y_norm / sdy

The parameters include those set for the Bayesian Tree Ensemble and those specific to the specification to estimate the heterogeneous treatment effect. 

The first important insight is that the specifcation of the causal model is a sum of two parts. The first part is a BART to estimate the expected outcome from a set of covariates and, if required, an estimate of the probability to receive treatment. The parameters for this BART model are denoted with `pr` for 'prognostic', e.g. `num_trees_pr`.    
The second part is a BART model that estimates the treatment effect conditional on the same or a different set of covariates, with its parameters denoted by `trt` as in 'treatment'.

The ensemble for the treatment effect may be a bit smaller/more restricted. 

Parameters that control the BART estimation:
- `num_cutpoints`: Number of cutpoints that are tested as splits for continuous covariates
- `num_sweeps` (and `burnin`): 

Parameters that control the size of the BART ensemble:
- `num_trees`: Number of trees in the ensemble. Probably somewhere between 50 and 250. 
- `Nmin`: Minimum number of samples in the final leaves
- `alpha`, `beta`: Control the tree depth by setting a prior for the probability that the current leaf is a final leaf formalized by $p(\text{leaf at depth t is not final leaf}) = \alpha(1+d)^{-\beta}$. A lower `alpha` and higher `beta` make the trees more shallow. Chipman et al. (2010) suggest $\alpha=0.95, \beta=2$. `alpha` is probably reasonable between [0.5,0.95] and `beta` between [1, 2].
- `tau`: Prior on the variance in the leaf for each tree. Hahn et al. propose to scale the prior with some factor the variance of the outcome divided by the number of trees. The factor might be somewhere between 0.1 and 10 but I have very little intuition there. 
- `mtry`: Number of variables to try at each split. Probably between half and all of them, closer to all of them. 

I believe the important parameters are `alpha`, `beta`,`tau`. In particular for `tau` I don't have a good intuition. 
Maybe important are `num_trees`, `num_sweeps`, `mtry`. 

**Please help me by tuning the parameters until we find a reasonable good estimate where the actual treatment effect distribution and the predicted treatment effects are close**. in particular, i've seen many results where the predicted effects don't reach fully down into the negative or positive effects. For example, the many negative effects are between -1 and -10 but the smallest predicted negative effect would be -1. 

Tuning using grid search would be nice but structured manual tuning might be sufficient. in the latter case, please log the results somewhere so we can compare them afterwards. 

In [8]:
from itertools import product
def make_grid(iterables):
    """
    Create a list of tuples from the combinations of elements in several lists of different length

    Output
    ------
    list of tuples or list of dicts
      if iterables is a dictionary of lists, the output is a list of dictionaries with the same keys and 
      values of each combination, e.g. {"A":[1,2], "B":[3]} -> [{"A":1, "B":3}, {"A":2, "B":3}]
    """
    if isinstance(iterables,dict):
        out = list(product(*iterables.values()))
        out = [dict(zip(iterables.keys(), x)) for x in out]
    else:
        out = list(product(*iterables))
 
    return out

In [9]:
n_categorical_vars = int(X.shape[1] - len(num_columns))

In [10]:
params = {
    'tau_pr': [0.1, 1, 10, 100],
    'tau_trt':[0.1, 1, 10, 100],
    'alpha': [0.95],
    'beta': [1.5, 2],
    'num_trees': [50, 100, 200],
    'num_sweeps': [100],
}

param_grid = make_grid(params)

In [11]:
len(param_grid)

96

In [12]:
z, z_val = z.astype(np.int32), z_val.astype(np.int32)

In [13]:
d_pr = X.shape[1] # Number of vars to consider in each split
d_trt = X.shape[1]

In [ ]:
%%time

BURNIN = 20 # Drop some samples in a warmup period of the sampler

results = []
pred = []
for i, param_set in enumerate(param_grid):
    cf = XBCF(
        #model="Normal",
        parallel=True, 
        #p_categorical_pr= n_categorical_vars,
        #p_categorical_trt= n_categorical_vars,
          num_sweeps=param_set['num_sweeps'], 
          burnin=BURNIN,
          max_depth=250,
            num_trees_pr=param_set['num_trees'],
            num_trees_trt=param_set['num_trees'],
            num_cutpoints=100,
            mtry_pr=d_pr,
            mtry_trt=d_trt,
            Nmin=1,
            tau_pr =  param_set['tau_pr']*np.var(y)/param_set['num_trees'],
            tau_trt = param_set['tau_trt']*np.var(y)/param_set['num_trees'], 
            no_split_penality="auto",
            alpha_pr= 0.95, # shrinkage (splitting probability)
            beta_pr= 2, # shrinkage (tree depth)
            alpha_trt= param_set['alpha'], # shrinkage for treatment part
            beta_trt= param_set['beta'],
             )
    
    cf = cf.fit(
    x_t=X, # Covariates treatment effect
    x=X, # Covariates outcome (including propensity score)
    y=y_norm,  # Outcome
    z=z, # Treatment group
    #p_cat=n_categorical_vars
    )
    
    tauhats_test = cf.predict(X)

    # Due to the scaling of y, the y and tau predictions need to be unscaled

    b = cf.b.transpose()
    a = cf.a.transpose()

    thats = sdy * cf.tauhats * (b[1] - b[0])
    thats_mean = np.mean(thats[:, BURNIN:], axis=1)
    yhats = cf.muhats * a + cf.tauhats * (b[1] - b[0])
    yhats_mean = np.mean(yhats[:, BURNIN:], axis=1)
    
    param_set['error'] = np.mean(abs(tau_response - thats_mean))
    
    results.append(param_set)
    pred.append(thats_mean)
    
    if i%10==0:
        print("Done iteration",i)

In [ ]:
results = pd.DataFrame(results)
results.to_csv("../results/xbcf_tuning_results.csv")

In [ ]:
pd.options.display.max_rows=999

In [ ]:
results

In [ ]:
results.iloc[results.error.idxmin(),:]

In [33]:
np.var(y_norm)

0.9999999999999999

In [14]:
class myXBCF(XBCF):
    def fit(self, x_t, x, y, z, p_cat=0):
        z= z.astype('int32')

        self.sdy = np.std(y)
        y = y - np.mean(y)
        y = y / self.sdy
        self.y = y

        super().fit(x_t, x, y, z, p_cat=p_cat)

        return self


    def predict(self, *args,**kwargs):
        tauhats = super().predict(*args,**kwargs)
        
        b = self.b.transpose()
        a = self.a.transpose()

        thats = self.sdy * tauhats * (b[1] - b[0])
        thats_mean = np.mean(thats[:, self.get_params()['burnin']:], axis=1)
        
        return thats_mean

In [34]:
d_pr

61

In [15]:
my_cf = myXBCF(random_seed=123, set_random_seed=True,
    #model="Normal",
    parallel=True, 
    #p_categorical_pr= n_categorical_vars,
    #p_categorical_trt= n_categorical_vars,
      num_sweeps=40, 
      burnin=20,
      max_depth=250,
        num_trees_pr=100,
        num_trees_trt=50,
        num_cutpoints=100,
        mtry_pr=d_pr,
        mtry_trt=d_trt,
        Nmin=50,
        tau_pr =  0.1/50,
        tau_trt = 0.1/50, 
        no_split_penality="auto",
        alpha_pr= 0.95, # shrinkage (splitting probability)
        beta_pr= 2, # shrinkage (tree depth)
        alpha_trt= 0.95, # shrinkage for treatment part
        beta_trt= 2,
         )

In [31]:
%%time
my_cf.fit(
x_t=X, # Covariates treatment effect
x=X, # Covariates outcome (including propensity score)
y=y,  # Outcome
z=z, # Treatment group
p_cat=1.*n_categorical_vars
)

TypeError: in method 'XBCFcpp__fit', argument 12 of type 'size_t'

In [18]:
%%time
cf = XBCF(random_seed=123,set_random_seed=True, 
    #model="Normal",
    parallel=True, 
    #p_categorical_pr= n_categorical_vars,
    #p_categorical_trt= n_categorical_vars,
      num_sweeps=40, 
      burnin=20,
      max_depth=250,
        num_trees_pr=10,
        num_trees_trt=10,
        num_cutpoints=20,
        mtry_pr=d_pr,
        mtry_trt=d_trt,
        Nmin=50,
        tau_pr =  0.1*np.var(y)/20,
        tau_trt = 0.1*np.var(y)/20, 
        no_split_penality="auto",
        alpha_pr= 0.95, # shrinkage (splitting probability)
        beta_pr= 2, # shrinkage (tree depth)
        alpha_trt= 0.95, # shrinkage for treatment part
        beta_trt= 2,
         )

cf.fit(
x_t=X, # Covariates treatment effect
x=X, # Covariates outcome (including propensity score)
y=y_norm,  # Outcome
z=z, # Treatment group
#p_cat=n_categorical_vars
)

CPU times: user 1min 45s, sys: 8.08 s, total: 1min 53s
Wall time: 29.5 s


XBCF(num_sweeps = 40, burnin = 20, max_depth = 250, Nmin = 50, num_cutpoints = 20, no_split_penality = 2.995732273553991, mtry_pr = 61, mtry_trt = 61, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 10, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 4.899426354183893, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 10, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 4.899426354183893, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = True, random_seed = 123, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [19]:
tauhats = cf.predict(X_val)

# Due to the scaling of y, the y and tau predictions need to be unscaled

b = cf.b.transpose()
a = cf.a.transpose()

thats = sdy * tauhats * (b[1] - b[0])
thats_mean = np.mean(thats[:, 5:], axis=1)
yhats = cf.muhats * a + cf.tauhats * (b[1] - b[0])
yhats_mean = np.mean(yhats[:, 5:], axis=1)

In [20]:
my_thats_mean = my_cf.predict(X_val)

In [21]:
np.corrcoef([thats_mean, my_thats_mean])

array([[1.        , 0.98262399],
       [0.98262399, 1.        ]])

In [24]:
my_cf.a.mean()

0.7589389223443418

In [25]:
cf.a.mean()

0.7589389223443418

In [22]:
sdy

31.30311918701998

In [23]:
my_cf.sdy

31.30311918701998

For some parameters the sampling fails and the internal results are NaN

We want the difference between `tau_response` an `thats_mean` to be as small as possible for the validation set

In [ ]:
np.sum(abs(tau_response - thats_mean))

In [ ]:
tau_response.mean(),  thats_mean.mean()

In [ ]:
plt.scatter(tau_response, thats_mean);

In [ ]:
tau_response.min()

In [ ]:
import seaborn as sns

In [ ]:
sns.kdeplot(tau_response)
sns.kdeplot(thats_mean)

In [ ]:
yhats = cf.muhats * a + cf.tauhats * (b[1] - b[0])
yhats_mean = np.mean(yhats[:, (BURNIN) :], axis=1)
plt.scatter(y[c==1], yhats_mean[c==1])

In [ ]:
y.mean(), yhats.mean()